In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load pre-trained FinBERT
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

class FinBERTAnalyzer:
    def __init__(self, model_name="ProsusAI/finbert"):
        """Load pre-trained FinBERT."""
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)
        self.label_map = {0: 'positive', 1: 'negative', 2: 'neutral'}

    def analyze_sentiment_zero_shot(self, text):
        """
        Get sentiment prediction without fine-tuning.
        Returns: {predicted_label, confidence, probabilities}
        """
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)

        probabilities = torch.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(probabilities).item()
        confidence = probabilities[0][predicted_class].item()

        return {
            'text': text,
            'predicted_label': self.label_map[predicted_class],
            'predicted_class': predicted_class,
            'confidence': confidence,
            'probabilities': {
                'positive': probabilities[0][0].item(),
                'negative': probabilities[0][1].item(),
                'neutral': probabilities[0][2].item()
            }
        }

    def analyze_batch(self, texts):
        """Efficiently analyze multiple texts."""
        results = []
        for text in texts:
            results.append(self.analyze_sentiment_zero_shot(text))
        return results

In [ ]:
obj=FinBERTAnalyzer()
texts=['Strong earnings growth beat expectations','Revenue declined significantly',
       'Results were in line with guidance','Interest rates of banks have hiked'
       ]
results=obj.analyze_batch(texts)
for r in results:
  for k,v in r.items():
    print(k,':\t',v)
  print('\n')


text :	 Strong earnings growth beat expectations
predicted_label :	 positive
predicted_class :	 0
confidence :	 0.9524754881858826
probabilities :	 {'positive': 0.9524754881858826, 'negative': 0.016565857455134392, 'neutral': 0.030958645045757294}


text :	 Revenue declined significantly
predicted_label :	 negative
predicted_class :	 1
confidence :	 0.9698547124862671
probabilities :	 {'positive': 0.011657973751425743, 'negative': 0.9698547124862671, 'neutral': 0.018487242981791496}


text :	 Results were in line with guidance
predicted_label :	 positive
predicted_class :	 0
confidence :	 0.6232041120529175
probabilities :	 {'positive': 0.6232041120529175, 'negative': 0.04456022381782532, 'neutral': 0.3322356641292572}


text :	 Interest rates of banks have hiked
predicted_label :	 positive
predicted_class :	 0
confidence :	 0.7887623310089111
probabilities :	 {'positive': 0.7887623310089111, 'negative': 0.12790566682815552, 'neutral': 0.08333207666873932}




In [10]:
print(obj.model.config.id2label)

{0: 'positive', 1: 'negative', 2: 'neutral'}
